# 목표 Multi Q-table
order box 타겟(A, B, ..., Q)에 대한 q-table을 q-learning으로 수렴시키게 만든다.   
총 18개(A, B, ..., Q, [9,4] = start point)의 수렴된 table을 만들어야 한다.   
order box를 얻은 경우 다음 order box를 향한다.    
start -> A -> B -> end로 움직이는 알고리즘을 짠다.

### 바닥부터 배우는 강화학습 챕터 6 Q-learning
책에 있는 코드를 참고해서 q-table이 제대로 수렴하는지 확인해보자.

In [ ]:
import random
import numpy as np
import sys

In [ ]:
class GridWorld():
    def __init__(self):
        self.x = 9
        self.y = 4

    def step(self, a):
        # 0번 액션: 왼쪽, 1번 액션: 위, 2번 액션: 오른쪽, 3번 액션 : 아래쪽
        if a==0:
            self.move_left()
        elif a==1:
            self.move_up()
        elif a==2:
            self.move_right()
        elif a==3:
            self.move_down()
    
        reward = -1
        done = self.is_done()
        return (self.x, self.y), reward, done

    def move_left(self):
        if self.y==0:
            pass
        # elif self.y==3 and self.x in [3, 4, 5, 6]:
        #     pass
        # elif self.y==5 and self.x in [3, 4, 5, 6]:
        #     pass
        # elif self.y==7 and self.x in [3, 4, 5, 6]:
        #     pass
        else:
            self.y -=1

    def move_right(self):
        if self.y==9:
            pass
        # elif self.y==1 and self.x in [3, 4, 5, 6]:
        #     pass
        # elif self.y==3 and self.x in [3, 4, 5, 6]:
        #     pass
        # elif self.y==5 and self.x in [3, 4, 5, 6]:
        #     pass
        else:
            self.y +=1

    def move_up(self):
        if self.x==0:
            pass
        # elif self.x==7 and self.y in [2, 4, 6]:
        #     pass
        else:
            self.x -= 1

    def move_down(self):
        if self.x==10:
            pass
        # elif self.x==1 and self.y in [2, 4, 6]:
        #     pass
        else:
            self.x+=1
    
    def is_done(self):
        if self.x==5 and self.y==0: # 목표
            return True
        else:
            return False
    
    def reset(self): #시작 state로 돌아가는 함수
        self.x = 9
        self.y = 4
        return (self.x, self.y)

In [ ]:
class QAgent():
    def __init__(self):
        self.q_table = np.zeros((10, 9, 4))
        self.eps = 0.5
    
    def select_action(self, s):
        x, y = s
        coin = random.random()
        if coin < self.eps:
            action = random.randint(0,3)
        else:
            action_val = self.q_table[x,y,:]
            action = np.argmax(action_val)
        return action

    def update_table(self, transition):
        s, a, r, s_prime = transition
        x, y = s
        next_x, next_y = s_prime
        a_prime = self.select_action(s_prime) 
        self.q_table[x,y,a] = self.q_table[x,y,a] + 0.1*(r + np.max(self.q_table[next_x, next_y, :]) - self.q_table[x,y,a])

    # def anneal_eps(self):
    #     self.eps -= 0.01 #q러닝에서는 epsilon이 더 천천히 줄어들도록 함(왜?)
    #     self.eps = max(self.eps, 0.2) # 0.2도 더 크다(왜?)

    def show_table(self):
        q_lst = self.q_table.tolist()
        data = np.zeros((10, 9))
        for row_idx in range(len(q_lst)):
#             print(row_idx)
            row = q_lst[row_idx]
#             print(q_lst)
            for col_idx in range(len(row)):
#                 print(f"col_idx = \n{col_idx}\n")
                col = row[col_idx]
                action = np.argmax(col)
                data[row_idx, col_idx] = action
        print(data)

In [ ]:
def main():
    env = GridWorld()
    agent = QAgent()
    episode = 10
    action_list = ['left','up','right','down']
    total_reward = 0

    for n_epi in range(episode): #10번 에피소드
        done = False
        s = env.reset()

        print(f"episode = {n_epi}")
        while not done: # 한 에피소드가 끝날때 까지
            a = agent.select_action(s)
            s_prime, r, done = env.step(a)
            agent.update_table((s, a, r, s_prime))
            s = s_prime
            total_reward += r

            print(f"a={action_list[a]}, s={s}, r={r}, total_reward={total_reward}\n")
            # print(agent.q_table)
            print("==================================")
        # agent.anneal_eps()
    agent.show_table()
    
# 좌, 상, 우, 하 0 1 2 3 
main()

수렴시킨 q-table의 값 저장 작업이 필요하다.

---

# 위의 수렴 과정 이후에 현재 task에 적용하기 아직 진행하지 않음

### Sim.py

In [ ]:
from string import ascii_uppercase
from draw_utils import *
from pyglet.gl import *
import numpy as np
import pandas as pd
import os



# reward
# move_reward = 0.1
# obs_reward = 0.1
# goal_reward = 10

move_reward = -1
obs_reward = -10
goal_reward = 50
print('reward:' , move_reward, obs_reward, goal_reward)

local_path = os.path.abspath(os.path.join(os.path.dirname(__file__)))


class Simulator:
    def __init__(self):
        '''
        height : 그리드 높이
        width : 그리드 너비 
        inds : A ~ Q alphabet list
        '''
        # Load train data
        self.files = pd.read_csv(os.path.join(local_path, "./data/factory_order_train.csv"))
        self.height = 10
        self.width = 9
        self.inds = list(ascii_uppercase)[:17]

    def set_box(self):
        '''
        아이템들이 있을 위치를 미리 정해놓고 그 위치 좌표들에 아이템이 들어올 수 있으므로 그리드에 100으로 표시한다.
        데이터 파일에서 이번 에피소드 아이템 정보를 받아 가져와야 할 아이템이 있는 좌표만 -100으로 표시한다.
        self.local_target에 에이전트가 이번에 방문해야할 좌표들을 저장한다.
        따라서 가져와야하는 아이템 좌표와 end point 좌표(처음 시작했던 좌표로 돌아와야하므로)가 들어가게 된다.
        '''
        box_data = pd.read_csv(os.path.join(local_path, "./data/box.csv"))

        # 물건이 들어있을 수 있는 경우
        for box in box_data.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(box, "row")][getattr(box, "col")] = 100

        # 물건이 실제 들어있는 경우
        order_item = list(set(self.inds) & set(self.items))
        order_csv = box_data[box_data['item'].isin(order_item)]

        for order_box in order_csv.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(order_box, "row")][getattr(order_box, "col")] = -100
            # local target에 가야 할 위치 좌표 넣기
            self.local_target.append(
                [getattr(order_box, "row"),
                 getattr(order_box, "col")]
                )

        self.local_target.sort()
        self.local_target.append([9,4]) 
        # 처음 부터 local_target을 [8,4]로 정해도 좋을듯하다.

        # 알파벳을 Grid에 넣어서 -> grid에 2Dconv 적용 가능

    def set_obstacle(self):
        '''
        장애물이 있어야하는 위치는 미리 obstacles.csv에 정의되어 있다. 이 좌표들을 0으로 표시한다.
        '''
        obstacles_data = pd.read_csv(os.path.join(local_path, "./data/obstacles.csv"))
        for obstacle in obstacles_data.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(obstacle, "row")][getattr(obstacle, "col")] = 0

    def reset(self, epi):
        '''
        reset()은 첫 스텝에서 사용되며 그리드에서 에이전트 위치가 start point에 있게 한다.

        :param epi: episode, 에피소드 마다 가져와야 할 아이템 리스트를 불러올 때 사용
        :return: 초기셋팅 된 그리드
        :rtype: numpy.ndarray
        _____________________________________________________________________________________
        items : 이번 에피소드에서 가져와야하는 아이템들
        terminal_location : 현재 에이전트가 찾아가야하는 목적지
        local_target : 한 에피소드에서 찾아가야하는 아이템 좌표, 마지막 엔드 포인트 등의 위치좌표들
        actions: visualization을 위해 에이전트 action을 저장하는 리스트
        curloc : 현재 위치
        '''

        # initial episode parameter setting
        self.epi = epi # 0~39999
        self.items = list(self.files.iloc[self.epi])[0] # item list
        self.cumulative_reward = 0
        self.terminal_location = None
        self.local_target = [] # items list
        self.actions = []

        # initial grid setting
        # 그림
        self.grid = np.ones((self.height, self.width), dtype="float16")

        # set information about the gridworld
        # set_box()는 local_target
        self.set_box()
        self.set_obstacle()

        # start point를 grid에 표시
        self.curloc = [9, 4]
        self.grid[int(self.curloc[0])][int(self.curloc[1])] = -5
        
        self.done = False
        
        return self.grid

    def apply_action(self, action, cur_x, cur_y):
        '''
        에이전트가 행한 action대로 현 에이전트의 위치좌표를 바꾼다.
        action은 discrete하며 4가지 up,down,left,right으로 정의된다.
        
        :param x: 에이전트의 현재 x 좌표
        :param y: 에이전트의 현재 y 좌표
        :return: action에 따라 변한 에이전트의 x 좌표, y 좌표
        :rtype: int, int
        '''
        new_x = cur_x
        new_y = cur_y
        # up
        if action == 0:
            new_x = cur_x - 1
        # down
        elif action == 1:
            new_x = cur_x + 1
        # left
        elif action == 2:
            new_y = cur_y - 1
        # right
        else:
            new_y = cur_y + 1

        return int(new_x), int(new_y)


    def get_reward(self, new_x, new_y, out_of_boundary):
        '''
        get_reward함수는 리워드를 계산하는 함수이며, 상황에 따라 에이전트가 action을 옳게 했는지 판단하는 지표가 된다.

        :param new_x: action에 따른 에이전트 새로운 위치좌표 x
        :param new_y: action에 따른 에이전트 새로운 위치좌표 y
        :param out_of_boundary: 에이전트 위치가 그리드 밖이 되지 않도록 제한
        :return: action에 따른 리워드
        :rtype: float
        '''

        # 바깥으로 나가는 경우
        if any(out_of_boundary):
            reward = obs_reward
                       
        else:
            # 장애물에 부딪히는 경우 
            if self.grid[new_x][new_y] == 0:
                reward = obs_reward  

            # 현재 목표에 도달한 경우
            elif new_x == self.terminal_location[0] and new_y == self.terminal_location[1]:
                reward = goal_reward

            # 그냥 움직이는 경우 
            else:
                reward = move_reward

        return reward

    def step(self, action):
        ''' 
        에이전트의 action에 따라 step을 진행한다.
        action에 따라 에이전트 위치를 변환하고, action에 대해 리워드를 받고, 어느 상황에 에피소드가 종료되어야 하는지 등을 판단한다.
        에이전트가 endpoint에 도착하면 gif로 에피소드에서 에이전트의 행동이 저장된다.

        :param action: 에이전트 행동
        :return:
            grid, 그리드
            reward, 리워드
            cumulative_reward, 누적 리워드
            done, 종료 여부
            goal_ob_reward, goal까지 아이템을 모두 가지고 돌아오는 finish율 계산을 위한 파라미터

        :rtype: numpy.ndarray, float, float, bool, bool/str

        (Hint : 시작 위치 (9,4)에서 up말고 다른 action은 전부 장애물이므로 action을 고정하는 것이 좋음)
        '''

        self.terminal_location = self.local_target[0]
        cur_x,cur_y = self.curloc
        self.actions.append((cur_x, cur_y))

        goal_ob_reward = False
        
        new_x, new_y = self.apply_action(action, cur_x, cur_y)

        out_of_boundary = [new_x < 0, new_x >= self.height, new_y < 0, new_y >= self.width]

        # 바깥으로 나가는 경우 종료
        if any(out_of_boundary):
            self.done = True
            goal_ob_reward = True
        else:
            # 장애물에 부딪히는 경우 종료
            if self.grid[new_x][new_y] == 0:
                self.done = True
                goal_ob_reward = True

            # 현재 목표에 도달한 경우, 다음 목표설정
            elif new_x == self.terminal_location[0] and new_y == self.terminal_location[1]:

                # end point 일 때
                if [new_x, new_y] == [9,4]:
                    self.done = True

                self.local_target.remove(self.local_target[0])
                self.grid[cur_x][cur_y] = 1
                self.grid[new_x][new_y] = -5
                goal_ob_reward = True
                self.curloc = [new_x, new_y]
            else:
                # 그냥 움직이는 경우 
                self.grid[cur_x][cur_y] = 1
                self.grid[new_x][new_y] = -5
                self.curloc = [new_x,new_y]
                
        reward = self.get_reward(new_x, new_y, out_of_boundary)
        self.cumulative_reward += reward

        if self.done == True:
            if [new_x, new_y] == [9, 4]:
                if self.terminal_location == [9, 4]:
                    # 완료되면 GIFS 저장
                    goal_ob_reward = 'finish'
                    height = 10
                    width = 9 
                    display = Display(visible=False, size=(width, height))
                    display.start()

                    start_point = (9, 4)
                    unit = 50
                    screen_height = height * unit
                    screen_width = width * unit
                    log_path = "./logs"
                    data_path = "./data"
                    render_cls = Render(screen_width, screen_height, unit, start_point, data_path, log_path)
                    for idx, new_pos in enumerate(self.actions):
                        render_cls.update_movement(new_pos, idx+1)
                    
                    render_cls.save_gif(self.epi, '_성공')
                    render_cls.viewer.close()
                    display.stop()
            else:     
                # 완료되면 GIFS 저장
                goal_ob_reward = 'finish'
                height = 10
                width = 9 
                display = Display(visible=False, size=(width, height))
                display.start()

                start_point = (9, 4)
                unit = 50
                screen_height = height * unit
                screen_width = width * unit
                log_path = "./logs"
                data_path = "./data"
                render_cls = Render(screen_width, screen_height, unit, start_point, data_path, log_path)
                for idx, new_pos in enumerate(self.actions):
                    render_cls.update_movement(new_pos, idx+1)

                render_cls.save_gif(self.epi, '_실패')
                render_cls.viewer.close()
                display.stop()
        return self.grid, reward, self.cumulative_reward, self.done, goal_ob_reward


if __name__ == "__main__":

    sim = Simulator()
    files = pd.read_csv("./data/factory_order_train.csv")
   
    for epi in range(2): # len(files)):
        items = list(files.iloc[epi])[0]
        done = False
        i = 0
        obs = sim.reset(epi)
        actions = [0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1]

        while done == False:
            
            i += 1
            next_obs, reward, cumul ,done, goal_reward = sim.step(actions[i])

            obs = next_obs

            if (done == True) or (i == (len(actions)-1)):
                i =0

### policy, action

In [ ]:
import random
import numpy as np
from Sim import Simulator

class Policy():
    def __init__(self):
        pass
    
    def select_action(self):
        pass
    
#     def predict_pi(self):
#         pass
    
#     def predict_value(self):
#         pass
    
#     def train_net(self):
#         pass

class RandomPolicy(Policy):
    def __init__(self):
        pass
    
    def select_action(self):
        coin = random.randint(0,3)
        return coin

# off-policy q-learning
class QLearning(Policy):
    def __init__(self):
#         self.q_table = np.zeros((sim.height, sim.width, 4), dtype="float16")
        self.q_table = np.zeros((10, 9, 4), dtype="float16")
        self.eps = 0.9
        self.alpha = 0.01
    
    def select_action(self, state):
        x, y = state
        coin = random.random()
        if coin < self.eps:
            action = random.randint(0,3)
        else:
            action_val = self.q_table[x,y,:]
            action = np.argmax(action_val)
            print(f"policy action = {action}")
        return action
    
    def update_table(self, history):
        s,a,r,s_prime = transition
        x, y = s
        next_x, next_y = s_prime
        # Q-learning update
        self.q_table[x,y,a] = self.q_table[x,y,a] + 0.1 * (r + np.amax(self.q_table[next_x,next_y,:]) - self.q_table[x,y,a])
        
    def anneal_eps(self):
        self.eps -= 0.01 # epsilon
        self.eps = max(self.eps, 0.2)
        
    def show_table(self):
        q_lst = self.q_table.tolist()
        data = np.zeors((10,9,4))
        for row_idx in range(len(q_lst)):
            row = q_lst[row_idx]
            for col_idx in range(len(row)):
                col = row[col_idx]
                action = np.argmax(col)
                data[row_idx, col_idx] = action
        print(data)
        

In [ ]:
import numpy as np
from Policy import *


class Agent():
    def __init__(self):
        self.target_list=None
        self.policy = None
        
    def set_target_list(self, local_target):
        self.target_list = local_target
        print("Agent's local target = {}".format(local_target))
    
    def reset(self):
        pass
    
    def update_target(self):
        pass
    
    # 현재 state를 받고 policy의 action하기
    # action_list = ['up','down','left','right'] = 0,1,2,3
    def select_action(self, state):
        # 진열대 array rack(좌 상 우, 세 종류) 만들어서 isin으로 체크하고 위치에 따라 행동 return 만들기
        left_rack = [5,0], [4,0], [3,0], [2,0]
        top_rack = [0,0], [0,1], [0,2], [0,3], [0,4], [0,5], [0,6], [0,7], [0,8]
        right_rack = [2,8], [3,8], [4,8], [5,8]
        if state == [9,4]:
            return 0 # up
        elif state in top_rack:
            return 1 # down
        elif state in right_rack:
            return 2 # left
        elif state in left_rack:
            return 3 # right
        else:
            action = self.policy.select_action(state)
            return action
    
    def set_policy(self, mode):
        if mode=="random":
            self.policy = RandomPolicy()
        elif mode=="q_learning":
            self.policy = QLearning()
        else:
            print("no")

### main

In [ ]:
import pandas as pd
from Sim import Simulator
from Agent import Agent

if __name__ == "__main__":
    files = pd.read_csv("./data/factory_order_train.csv")
    
    sim = Simulator()
    agent = Agent()
    agent.set_policy('random')
   
    for epi in range(5): # len(files)):
        # env 초기화
        obs = sim.reset(epi)
        print('\n\nenv 초기화 완료, {}번째 에피소드 시작'.format(epi))
        
        # agent에게 local_target 주기
        items = list(files.iloc[epi])[0]
        agent.set_target_list(items)
        done = False
        
        while done == False:
            # agent 랜덤 행동 취하기
            action = agent.select_action()
            print(action)
            next_obs, reward, cumul ,done, goal_reward = sim.step(action)
            print('done', done)
            obs = next_obs

---

---

### 바닥부터 배우는 강화학습 챕터 6 Q-learning

In [1]:
import random
import numpy as np

In [2]:
class GridWorld(): # 그리드월드 클래스 생성
    def __init__(self): # 초기 변수 x, y (그리드 월드 시작 state)
        self.x = 0
        self.y = 0

    def step(self, a):
        # 0번 액션: 왼쪽, 1번 액션: 위, 2번 액션: 오른쪽, 3번 액션 : 아래쪽
        if a==0:
            self.move_left() # 각 이동함수를 통해서 현재 state의 좌표를 변환한다.
        elif a==1:
            self.move_up()
        elif a==2:
            self.move_right()
        elif a==3:
            self.move_down()
    
        reward = -1 #보상은 1로 고정 (MDP를 모르므로 실제 관측하기 전까지는 모른다고 가정)
        done = self.is_done() # done을 불리언으로 판정 terminal state 판단을 위해
        return (self.x, self.y), reward, done

    def move_left(self):
        if self.y==0: #벽(테두리)지정
            pass
        elif self.y==3 and self.x in [0, 1, 2]: #장애물도 마찬가지로 지정해준다.
            pass
        elif self.y==5 and self.x in [2, 3, 4]:
            pass
        else:
            self.y -=1

    def move_right(self):
        if self.y==1 and self.x in [0, 1, 2]:
            pass
        elif self.y==3 and self.x in [2, 3, 4]:
            pass
        elif self.y==6:
            pass
        else:
            self.y +=1

    def move_up(self):
        if self.x==0:
            pass
        elif self.x==3 and self.y==2:
            pass
        else:
            self.x -= 1

    def move_down(self):
        if self.x==4:
            pass
        elif self.x==1 and self.y==4:
            pass
        else:
            self.x+=1
    
    def is_done(self):
        if self.x==4 and self.y==6: # 목표 state인 (4, 6)에 도달하면 끝난다.
            return True
        else:
            return False
    
    def reset(self): #시작 state로 돌아가는 함수
        self.x = 0
        self.y = 0
        return (self.x, self.y)

In [22]:
class QAgent():
    def __init__(self):
        self.q_table = np.zeros((5, 7, 4)) #q밸류를 저장하는 변수, 모두 0이고 3차원임
        self.eps = 0.9 
    
    def select_action(self, s):
        # 엡실론-그리디로 액션 선택
        x, y = s
        coin = random.random()
        if coin < self.eps:
            action = random.randint(0,3)
        else:
            print(self.q_table[x,y,:])
            action_val = self.q_table[x,y,:]
            action = np.argmax(action_val)
        return action

    def update_table(self, transition):
        s, a, r , s_prime = transition
        x, y = s
        next_x, next_y = s_prime
        a_prime = self.select_action(s_prime) 
        #SARSA대신 Q러닝 업데이트 식을 이용
        self.q_table[x,y,a] = self.q_table[x,y,a] + 0.1*(r + np.max(self.q_table[next_x, next_y, :]) - self.q_table[x,y,a])
        #공식에 따라서 SARSA에서는 그냥 다음 액션의 Q_table을 가져오던 것에서 다음 Q_table의 max를 가져오는것으로 바뀜

    def anneal_eps(self):
        self.eps -= 0.01 #q러닝에서는 epsilon이 더 천천히 줄어들도록 함(왜?)
        self.eps = max(self.eps, 0.2) # 0.2도 더 크다(왜?)

    def show_table(self):
        q_lst = self.q_table.tolist()
        print(self.q_table)
        data = np.zeros((5, 7))
        for row_idx in range(len(q_lst)):
#             print(row_idx)
            row = q_lst[row_idx]
#             print(q_lst)
            for col_idx in range(len(row)):
#                 print(f"col_idx = \n{col_idx}\n")
                col = row[col_idx]
                action = np.argmax(col)
                data[row_idx, col_idx] = action
        print(data)

In [23]:
def main():
    env = GridWorld()
    agent = QAgent()
    episode = 1000

    for n_epi in range(episode): #1000번 에피소드
        done = False

        s = env.reset()
        while not done: # 한 에피소드가 끝날때 까지
            a = agent.select_action(s)
            s_prime, r, done = env.step(a)
            agent.update_table((s, a, r, s_prime))
            s = s_prime
        agent.anneal_eps()

    agent.show_table()
    
# 좌, 상, 우, 하
main()

[[[-14.93967852 -14.99143922 -13.99999999 -13.99999999]
  [-14.44717042 -13.79653821 -13.82424545 -13.        ]
  [  0.           0.           0.           0.        ]
  [ -6.93439226  -6.93736874  -6.82738185  -6.83894806]
  [ -6.08550143  -6.24658731  -5.98089047  -5.98016746]
  [ -5.79734961  -5.29701372  -4.99502375  -4.99482916]
  [ -4.60161566  -4.19239284  -4.11643606  -3.99970726]]

 [[-13.94981503 -14.94345962 -12.99999999 -12.99999999]
  [-13.74811209 -13.90794335 -12.89308869 -12.        ]
  [  0.           0.           0.           0.        ]
  [ -6.99487715  -7.68089704  -6.          -7.99149948]
  [ -6.99029617  -6.94796811  -5.          -5.99268867]
  [ -5.99427936  -5.94492815  -4.          -4.        ]
  [ -4.6964982   -4.59569735  -3.79804845  -3.        ]]

 [[-12.87278989 -13.84571914 -12.         -12.        ]
  [-12.94861796 -12.95377784 -11.98798441 -11.        ]
  [  0.           0.           0.           0.        ]
  [ -7.99378792  -7.          -7.99928517  -

In [9]:
main()

[[3. 3. 0. 2. 2. 3. 3.]
 [3. 3. 0. 2. 2. 3. 3.]
 [3. 3. 0. 1. 0. 3. 3.]
 [2. 2. 2. 1. 0. 3. 3.]
 [3. 1. 2. 1. 0. 2. 0.]]
